In [ ]:
#@markdown ## **Don't change any of these values.**
#-----------------------------------------------------
# @markdown #### `Required fields:`
project_id = 1  #@param {type: 'integer'}
weights = ''  #@param {type: 'string'}
#-----------------------------------------------------
#@markdown #### `Optional fields:`
tasks_range = ''  #@param {type: 'string'}
#@markdown > **Usage:** If you specify a `task_range`, the model will only predict tasks within that range. **Example usage:** `20000,21000`. **Default:** leave empty.
predict_all = False  #@param {type: 'boolean'}
#@markdown > **Usage:** If you check this box, the model will predict **ALL** the tasks in the project. **Default:** False (unchecked).
model_version = ''  #@param {type: 'string'}
debug = False  #@param #@param {type: 'boolean'}
one_task = False  # integer
multithreading = True  #@param {type: 'boolean'}
delete_if_no_predictions = False  # @param {type: 'boolean'}
if_empty_apply_label = 'no animal'  # @param {type: 'string'}
#-----------------------------------------------------
# @markdown #### `W&B variables:`s
wandb_entity = ''  #@param {type: 'string'}
wandb_project = ''  #@param {type: 'string'}


**Run this command on the local server:**


```shell
croc send "BirdFSD-YOLOv5/.env"
```

- Copy the passphrase part and paste it in the cell input below.


In [ ]:
!rm -rf /root/.cache

! git clone --quiet https://github.com/bird-feeder/BirdFSD-YOLOv5.git
%cd BirdFSD-YOLOv5
! pip -q install -r requirements.txt > /dev/null 2>&1
! pip -q install "PyYAML>=5.3.1" wandb > /dev/null 2>&1
! pip -q install "pymongo[srv]>=4.0.2"

! curl https://getcroc.schollz.com | bash > /dev/null 2>&1

PASSPHRASE = '' #@param {type:"string"}
if not PASSPHRASE:
    raise Exception('Paste the passphrase in the input bar!')
if 'croc' in PASSPHRASE:
    PASSPHRASE = PASSPHRASE.split('croc ')[1]

! croc --yes $PASSPHRASE > /dev/null 2>&1
! wget "$weights" -O best.pt
weights = "best.pt"
# ! export $(cat .env | xargs) && python utils/best_weights.py -e "$wandb_entity" -p "$wandb_project"

! mkdir ultralytics
! git clone --quiet https://github.com/ultralytics/yolov5.git ultralytics/yolov5

print('\n>>>> Clear the passphrase input field!')

In [ ]:
import os

from dotenv import load_dotenv
from predict import Predict
from mongodb_helper import get_tasks_from_mongodb

load_dotenv()
get_tasks_from_mongodb(project_id)

predict = Predict(weights, project_id, tasks_range, predict_all, one_task,
                  model_version, multithreading, delete_if_no_predictions,
                  if_empty_apply_label, debug)

try:
    predict.apply_predictions()
    print('\n\nFinished successfully!')
    print('Clear the output: `Edit` -> `Clear all outputs`.')
except KeyboardInterrupt:
    print('Interrupted...')
finally:
    os.remove('tasks.json')
